In [3]:
import numpy as np
from Helpers import *
from mainSVT import *
import time
import random
import matplotlib.pyplot as plt
import scipy.sparse as sp
from sklearn.decomposition import NMF


In [4]:

# Set random seed for reproducibility
np.random.seed(42)

# Load the data from the specified .npz file
file = np.load(".\\data\\2024-03-29-data.npz")

# Extract the necessary components from the file
data = file['data']               # Raw Data
mz_values = file['mz_values']     # Corresponding mz-values
image_size = file['image_size']   # Size of the image
mz_picks = file['mz_picks']       # Selected mz-ids
num = file['num']                 # Number of picks

In [6]:
# 
def create_sparse_random_matrix(rank, rows, cols, density=0.1):
    """
    Create a sparse random matrix with given rank and dimensions.

    Parameters:
    rank (int): The rank of the matrix.
    rows (int): The number of rows of the matrix.
    cols (int): The number of columns of the matrix.
    density (float): The density of the sparse matrix.

    Returns:
    scipy.sparse.csr_matrix: The sparse random matrix.
    """
    # Generate two random matrices for low-rank decomposition
    A = np.random.randn(rows, rank)
    B = np.random.randn(rank, cols)

    # Create the low-rank matrix
    low_rank_matrix = np.dot(A, B)

    # Create a sparse mask
    mask = sp.random(rows, cols, density=density, format='csr', data_rvs=np.ones).astype(bool)

    # Apply the mask to the low-rank matrix
    sparse_matrix = sp.csr_matrix(low_rank_matrix * mask.toarray())

    return sparse_matrix


def calculate_sparsity(matrix):
    """
    Calculate the sparsity of a matrix.

    Parameters:
    matrix (numpy.ndarray or scipy.sparse.spmatrix): The matrix.

    Returns:
    float: The sparsity percentage of the matrix.
    """
    # Convert numpy matrix to sparse matrix if necessary
    if isinstance(matrix, np.ndarray):
        matrix = sp.csr_matrix(matrix)

    # Calculate total elements, zero elements, and sparsity
    total_elements = matrix.shape[0] * matrix.shape[1]
    zero_elements = total_elements - matrix.nnz
    sparsity = (zero_elements / total_elements) * 100
    
    return sparsity


def massbinImageComparisonGPT(massbin_of_interest, data, sampled_data, reconstructed_data, colormap='cubehelix'):
    colormaps = [colormap]*3
    
    data_matrix = data[:, massbin_of_interest].reshape((500, 100))
    sampled_data_matrix = sampled_data[:, massbin_of_interest].reshape((500, 100))
    reconstructed_data_matrix = reconstructed_data[:, massbin_of_interest].reshape((500, 100))
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))  # Create a figure with 1 row and 3 columns
    
    # Plot each matrix with its respective colormap
    min_val = np.min(data_matrix)
    max_val = np.max(data_matrix)
    for ax, matrix, cmap, title in zip(axes, [data_matrix, reconstructed_data_matrix,sampled_data_matrix], colormaps, ['Original Data', 'Reconstructed Data', 'Sampled Data']):
        cax = ax.imshow(matrix, aspect='equal', cmap=cmap, vmin=min_val, vmax=max_val)  # Set aspect to 'equal' and use same limits as original data
        ax.set_title(f'{title} Colormap')
        fig.colorbar(cax, ax=ax)
    
    # Add overall title and show the plot
    fig.suptitle(f'Side by Side Comparison of Data, Reconstruction and Sampled Data of Massbin {massbin_of_interest}', fontsize=16)
    plt.show() 
    
def normalizeData(data):
    # Normalize the data
    row_max = data.max(axis=1)
    n_data = data / row_max[:, np.newaxis]
    n_data = n_data.T
    return n_data, row_max

def denormalizeData(n_data, row_max):
    # Denormalize the data
    denormalized_data = n_data.T * row_max[:, np.newaxis]
    return denormalized_data


def run_nmf_and_compute_error(data, keep_ratio, tolerance=1e-4, max_iter=150, rank=50):
    data_sampled, coords, coords_TF, M_sampled_nans = sample_from_matrix(data, ratio_to_keep=keep_ratio, seed=random.randint(0,2000))
    n_data_sampled, row_max = normalizeData(data_sampled)
    
    model = NMF(rank, init="random", solver='mu', max_iter=100, random_state=42)
    
    # Fit the model to the data
    W = model.fit_transform(n_data_sampled)

    # Get the H matrix
    H = model.components_
    
    X_n_opt = np.dot(W,H)
    
    spars = calculate_sparsity(H)

    X_reconstructed = denormalizeData(X_n_opt, row_max)
    print(spars)
    
    
    g_error = general_error_relative(data, X_reconstructed)
    is_error = insampling_error_relative(data_sampled, X_reconstructed, coords_TF)
    os_error = out_of_sample_error_relative(data, X_reconstructed, coords_TF)
    
    return g_error, is_error, os_error, spars, X_n_opt

def average_general_error(num_runs, rank, keep_ratio):
    tolerance = 1e-4
    max_iter = 100
    
    g_errors = []
    is_errors = []
    os_errors = []
    spars_l = []
    start = time.time()
    for _ in range(num_runs):
        print('\n')
        print('\n')
        print('\n')
        print('RUNNING',_)
        print('\n')
        print('\n')
        print('\n')
        
        g_error, is_error, os_error, spars, X_n_opt = run_nmf_and_compute_error(data, keep_ratio, tolerance, max_iter, rank)
        
        g_errors.append(g_error)
        is_errors.append(is_error)
        os_errors.append(os_error)
        spars_l.append(spars)
        
    avg_time =  (time.time()-start)/num_runs
    avg_spars = np.mean(spars_l)
    avg_g_error = np.mean(g_errors)
    avg_is_error = np.mean(is_errors)
    avg_os_error = np.mean(os_errors)
    
    print('\n \n')
    print(f"At rank {rank} and keep ratio {keep_ratio}:")
    print(f"Average General Error: {100*avg_g_error}%")
    print(f"Average In-Sample Error: {100*avg_is_error}%")
    print(f"Average Out-of-Sample Error: {100*avg_os_error}%")
    print(f"Average Sparsity: {avg_spars}%")
    print(f"Average Time: {avg_time:.2f} seconds")
    print(f"Average Iterations: {max_iter:.2f}")
    print("\n")
    
    return avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time, X_n_opt



2.260112

In [11]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time = average_general_error(num_runs=5, rank=10, keep_ratio=0.5)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 1








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 2








...

0.0






RUNNING 3








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 4








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 10 and keep ratio 0.5:
Average General Error: 53.023019541580126%
Average In-Sample Error: 51.303041793626356%
Average Out-of-Sample Error: 54.68695682678286%
Average Sparsity: 0.0%
Average Time: 10.56 seconds
Average Iterations: 100.00




In [12]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time = average_general_error(num_runs=5, rank=10, keep_ratio=0.3)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 1








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 2








...

0.0






RUNNING 3








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 4








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 10 and keep ratio 0.3:
Average General Error: 71.84106057505136%
Average In-Sample Error: 68.75822705878866%
Average Out-of-Sample Error: 73.11904117088176%
Average Sparsity: 0.0%
Average Time: 9.88 seconds
Average Iterations: 100.00




In [13]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time = average_general_error(num_runs=5, rank=25, keep_ratio=0.5)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 1








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 2








...

0.0






RUNNING 3








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 4








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 25 and keep ratio 0.5:
Average General Error: 54.938390609668296%
Average In-Sample Error: 47.285606150882025%
Average Out-of-Sample Error: 61.6363469077499%
Average Sparsity: 0.0%
Average Time: 12.82 seconds
Average Iterations: 100.00




In [14]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time = average_general_error(num_runs=5, rank=25, keep_ratio=0.3)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 1








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 2








...

0.0






RUNNING 3








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 4








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 25 and keep ratio 0.3:
Average General Error: 73.3779562504368%
Average In-Sample Error: 61.217725198506535%
Average Out-of-Sample Error: 78.00212263464468%
Average Sparsity: 0.0%
Average Time: 11.92 seconds
Average Iterations: 100.00




In [15]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time = average_general_error(num_runs=5, rank=50, keep_ratio=0.5)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 1








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 2








...

0.0






RUNNING 3








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 4








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 50 and keep ratio 0.5:
Average General Error: 57.20941215185196%
Average In-Sample Error: 43.57834326869502%
Average Out-of-Sample Error: 68.17304854327135%
Average Sparsity: 0.0%
Average Time: 16.91 seconds
Average Iterations: 100.00




In [16]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time = average_general_error(num_runs=5, rank=50, keep_ratio=0.3)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 1








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 2








...

0.0






RUNNING 3








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0






RUNNING 4








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 50 and keep ratio 0.3:
Average General Error: 75.37039298281643%
Average In-Sample Error: 55.41768875640719%
Average Out-of-Sample Error: 82.43700084222837%
Average Sparsity: 0.0%
Average Time: 15.32 seconds
Average Iterations: 100.00




In [15]:
avg_g_error, avg_is_error, avg_os_error, avg_spars, avg_time, X_n_opt = average_general_error(num_runs=1, rank=50, keep_ratio=0.5)







RUNNING 0








C:\Users\kesol\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


0.0

 

At rank 50 and keep ratio 0.5:
Average General Error: 57.21993602384988%
Average In-Sample Error: 42.7420849064751%
Average Out-of-Sample Error: 68.74160300131203%
Average Sparsity: 0.0%
Average Time: 20.57 seconds
Average Iterations: 100.00




In [16]:
np.savez_compressed('Out/MU_50', data=X_n_opt)